In [15]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot
import numpy as np
path=r"D:\New folder (5)\Video.mp4"
cap = cv2.VideoCapture(path)
detector = FaceMeshDetector(maxFaces=1)
plotY = LivePlot(640, 360, [20, 50], invert=True)
idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
ratioList = []
blinkCounter = 0
counter = 0
color = (255, 0, 0)
blank = np.zeros((500, 500, 1), dtype = "uint8")
lable=[]
while True:
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    success, img = cap.read()
    #img=cv2.imread(r'D:\DWONLAOD\dwon\sample-result-1.png')
    img, faces = detector.findFaceMesh(img, draw=True)
    if faces:
        face = faces[0]
        for id in idList:
            cv2.circle(img, face[id], 5,color, cv2.FILLED)
        leftUp = face[159]
        leftDown = face[23]
        leftLeft = face[130]
        leftRight = face[243]
        lenghtVer, _ = detector.findDistance(leftUp, leftDown)
        lenghtHor, _ = detector.findDistance(leftLeft, leftRight)
        cv2.line(img, leftUp, leftDown, (255, 200, 0), 3)
        cv2.line(img, leftLeft, leftRight, (0, 200, 0), 3)
        ratio = int((lenghtVer / lenghtHor) * 100)
        ratioList.append(ratio)
        if len(ratioList) > 3:
            ratioList.pop(0)
        ratioAvg = sum(ratioList) / len(ratioList)
        if 39<ratioAvg< 43 and counter == 0:
            blinkCounter += 1
            color = (0,200,0)
            counter = 1
        if counter != 0:
            counter += 1
            if counter > 5:
                counter = 0
                color = (255,0, 255)
        cvzone.putTextRect(img, f'Blink Count: {blinkCounter}', (50, 100),
                           colorR=color)
        if 5<blinkCounter<10:
            cvzone.putTextRect(img, 'real ', (50, 150),
                   colorR=color)
            lable.append('real')
        elif blinkCounter>10 :
            blinkCounter=0
        imgPlot = plotY.update(ratioAvg, color)
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, imgPlot], 2, 1)
    else:
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, blank], 2, 1)
    cv2.imshow("Image", imgStack)
    key=cv2.waitKey(30)
    if key == ord('q'):
        break
cap.release()        
cv2.destroyAllWindows()    


In [16]:
cv2.imshow('banlk',blank)
cv2.waitKey(0)


-1

## 